# ANCOVA

In [86]:
import os,sys
import numpy as np
import pandas as pd
import pylab as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

src_dir = os.path.abspath(os.pardir)
sys.path[0] = src_dir

from analysis.CodingScoring import scoreTIPI, normTIPI
plt.style.use('dark_background')

src_dir

'/Users/melissawessel/Documents/dev/python/personality-stratification'

In [14]:

questions = pd.read_csv('../data/External/dataHeadings.csv')
dataRaw = pd.read_csv('../data/External/dataRaw.csv', names=questions.QuestionCode)
dataRaw.head()

,Start Date,End Date,DressColor,DressColorOther,DressShadow,DressShadowOther,DressLight,DressTrueColors,DressTrueColorsOther,JacketColor,...,C+,S-,O+,E-,A+,C-,S+,O-,HonestyCheck,FreeText
0,10/14/17 0:50,10/14/17 0:59,White/Gold,NaN,Yes,NaN,Artificial light,No,NaN,Other (please specify),...,Neither agree nor disagree,Agree slightly,Agree strongly,Agree slightly,Agree strongly,Agree moderately,Agree slightly,Disagree moderately,Yes,Best of luck!
1,10/14/17 0:30,10/14/17 0:40,NaN,Switches from blue/black to blue/gold,No,NaN,Artificial light,No,NaN,Other (please specify),...,Agree strongly,Agree slightly,Agree slightly,Disagree slightly,Agree moderately,Disagree strongly,Agree moderately,Disagree strongly,Yes,Hope research is going well!! Good luck!
2,10/14/17 0:00,10/14/17 0:04,Blue/Black,NaN,No,NaN,Artificial light,No,NaN,Blue and white,...,Agree slightly,Agree slightly,Agree moderately,Agree slightly,Agree slightly,Agree slightly,Agree slightly,Agree slightly,Yes,NaN
3,10/13/17 23:38,10/13/17 23:45,White/Gold,NaN,Yes,NaN,Natural light,No,NaN,Green and gold,...,Agree moderately,Disagree moderately,Agree moderately,Disagree slightly,Disagree slightly,Disagree moderately,Agree moderately,Neither agree nor disagree,Yes,I'm an engineer and would love to see the data...
4,10/13/17 23:24,10/13/17 23:30,White/Gold,NaN,No,NaN,Natural light,No,NaN,Other (please specify),...,Agree slightly,Agree slightly,Neither agree nor disagree,Agree slightly,Agree moderately,Neither agree nor disagree,Agree slightly,Agree slightly,Yes,NaN


In [4]:
df = dataRaw.join(normTIPI(scoreTIPI(dataRaw)))

In [5]:
cxOCEAS = df[['Chronotype','O','C','E','A','S']]

In [6]:
cxOCEAS['N'] = 0-cxOCEAS['S']
df = cxOCEAS.drop(columns=['S'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df.head()

,Chronotype,O,C,E,A,N
0,Strong owl,0.871115,-1.163156,0.347041,-0.578990,0.242004
1,Strong owl,0.432133,1.664499,0.662877,-0.578990,-0.096085
2,Owl,-0.884811,-0.456242,0.031205,-0.986787,0.242004
3,Lark,-0.445830,0.957585,0.031205,-1.802382,-1.110352
4,Lark,-1.762774,-0.102785,-0.916303,0.644401,0.242004


In [8]:
df = df.dropna()
for trait in ['N','E','O','A','C']:
    df['%s_strat'%trait]=0
    df['%s_strat'%trait].loc[df[trait]<-1] = 1
    df['%s_strat'%trait].loc[df[trait]>1] = 3
    df['%s_strat'%trait].loc[df['%s_strat'%trait]==0] = 2
df.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Chronotype,O,C,E,A,N,N_strat,E_strat,O_strat,A_strat,C_strat
0,Strong owl,0.871115,-1.163156,0.347041,-0.578990,0.242004,2,2,2,2,1
1,Strong owl,0.432133,1.664499,0.662877,-0.578990,-0.096085,2,2,2,2,3
2,Owl,-0.884811,-0.456242,0.031205,-0.986787,0.242004,2,2,2,2,2
3,Lark,-0.445830,0.957585,0.031205,-1.802382,-1.110352,1,2,2,1,2
4,Lark,-1.762774,-0.102785,-0.916303,0.644401,0.242004,2,2,1,2,2


In [9]:
for trait in ['N','E','O','A','C']:
    df['%s_bistrat'%trait]=0
    df['%s_bistrat'%trait].loc[df[trait]<0] = 1
    df['%s_bistrat'%trait].loc[df[trait]>0] = 3
df.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Chronotype,O,C,E,A,N,N_strat,E_strat,O_strat,A_strat,C_strat,N_bistrat,E_bistrat,O_bistrat,A_bistrat,C_bistrat
0,Strong owl,0.871115,-1.163156,0.347041,-0.578990,0.242004,2,2,2,2,1,3,3,3,1,1
1,Strong owl,0.432133,1.664499,0.662877,-0.578990,-0.096085,2,2,2,2,3,1,3,3,1,3
2,Owl,-0.884811,-0.456242,0.031205,-0.986787,0.242004,2,2,2,2,2,3,3,1,1,1
3,Lark,-0.445830,0.957585,0.031205,-1.802382,-1.110352,1,2,2,1,2,1,3,1,1,3
4,Lark,-1.762774,-0.102785,-0.916303,0.644401,0.242004,2,2,1,2,2,3,1,1,3,1


In [10]:
df = df.replace(['Strong owl','Owl','Lark','Strong lark'],[1,2,3,4])

In [11]:
df.head()

,Chronotype,O,C,E,A,N,N_strat,E_strat,O_strat,A_strat,C_strat,N_bistrat,E_bistrat,O_bistrat,A_bistrat,C_bistrat
0,1,0.871115,-1.163156,0.347041,-0.578990,0.242004,2,2,2,2,1,3,3,3,1,1
1,1,0.432133,1.664499,0.662877,-0.578990,-0.096085,2,2,2,2,3,1,3,3,1,3
2,2,-0.884811,-0.456242,0.031205,-0.986787,0.242004,2,2,2,2,2,3,3,1,1,1
3,3,-0.445830,0.957585,0.031205,-1.802382,-1.110352,1,2,2,1,2,1,3,1,1,3
4,3,-1.762774,-0.102785,-0.916303,0.644401,0.242004,2,2,1,2,2,3,1,1,3,1


In [23]:
x = sm.add_constant(df['C'])
y = df['Chronotype']

In [24]:
reg = sm.OLS(y,x)
results = reg.fit()

This method doesn't divide into training and testing sets - r-squared is inflated because it's based on the data that trained the model

In [25]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Chronotype   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     881.1
Date:                Tue, 05 Feb 2019   Prob (F-statistic):          2.16e-188
Time:                        15:30:59   Log-Likelihood:                -19935.
No. Observations:               15539   AIC:                         3.987e+04
Df Residuals:                   15537   BIC:                         3.989e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1654      0.007    301.559      0.000       2.151       2.179
C              0.2154      0.007     29.684      0.000       0.201       0.230
==============================================================================
Omnibus:                      709.000   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              388.426
Skew:                           0.226   Prob(JB):                     4.51e-85
Kurtosis:                       2.371   Cond. No.                         1.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
x = sm.add_constant(df['C_bistrat'])

In [30]:
reg = sm.OLS(y,x)
results = reg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Chronotype   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     570.1
Date:                Tue, 05 Feb 2019   Prob (F-statistic):          8.98e-124
Time:                        15:34:01   Log-Likelihood:                -20084.
No. Observations:               15539   AIC:                         4.017e+04
Df Residuals:                   15537   BIC:                         4.019e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8441      0.017    108.628      0.000       1.811       1.877
C_bistrat      0.1709      0.007     23.877      0.000       0.157       0.185
==============================================================================
Omnibus:                      856.079   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              440.687
Skew:                           0.240   Prob(JB):                     2.02e-96
Kurtosis:                       2.330   Cond. No.                         6.56
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
.035/.054

0.6481481481481483

Try to do ANCOVA

In [76]:
df_combined = pd.DataFrame(df[['C_strat','Chronotype']])

In [77]:
df_combined['Condition'] = True

In [78]:
df_combined.rename(columns={'C_strat':'Consc'}, inplace=True)

In [79]:
df_combined

,Consc,Chronotype,Condition
0,1,1,True
1,3,1,True
2,2,2,True
3,2,3,True
4,2,3,True
5,2,2,True
6,2,2,True
7,2,3,True
8,2,2,True
9,3,1,True


In [80]:
df_combined_c = pd.DataFrame(df[['C','Chronotype']])

In [81]:
df_combined_c['Condition'] = False

In [82]:
df_combined_c.rename(columns={'C':'Consc'}, inplace=True)
df_combined = df_combined.append(df_combined_c)

In [83]:
df_combined

,Consc,Chronotype,Condition
0,1.000000,1,True
1,3.000000,1,True
2,2.000000,2,True
3,2.000000,3,True
4,2.000000,3,True
5,2.000000,2,True
6,2.000000,2,True
7,2.000000,3,True
8,2.000000,2,True
9,3.000000,1,True


In [85]:
formula = 'Chronotype ~ Consc * C(Condition)'  # ANCOVA formula
lm = ols(formula, df_combined)
fit = lm.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Chronotype   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     511.7
Date:                Tue, 05 Feb 2019   Prob (F-statistic):               0.00
Time:                        16:01:20   Log-Likelihood:                -39978.
No. Observations:               31078   AIC:                         7.996e+04
Df Residuals:                   31074   BIC:                         8.000e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      2.1654      0.007    300.514      0.000       2.151       2.179
C(Condition)[T.True]          -0.6160      0.028    -22.233      0.000      -0.670      -0.562
Consc                          0.2154      0.007     29.581      0.000       0.201       0.230
Consc:C(Condition)[T.True]     0.1053      0.014      7.289      0.000       0.077       0.134
==============================================================================
Omnibus:                     1530.846   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              810.721
Skew:                           0.228   Prob(JB):                    9.00e-177
Kurtosis:                       2.353   Cond. No.                         14.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Booooo, the difference between the coefficients of continuous and binned linear regressions is statistically significant (duh becaus